In [8]:
from FK import *
import pyvoro
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# import numba as nb
from tqdm import tqdm

## Define functions

In [9]:
# given vertices of polygon, evaluate the area
def area_vertices(vertices):
    f_cm = np.mean(vertices,axis=0)
    vertices = vertices - f_cm
    i_1 = np.arange(len(vertices))
    i_2 = (i_1+1)%len(i_1)
    A = [np.cross(vertices[i_1[j]],vertices[i_2[j]]) for j in range(len(i_1))]
    area = np.linalg.norm(np.sum(A,axis=0)/2)
    return area

# return the n unitvectors uniformly distributed on the sphere
def sphere_sample(n):
    samples = []
    for i in range(n):
        v = [0, 0, 0]  # initialize so we go into the while loop

        while np.linalg.norm(v) < .0001:
            x = np.random.normal()  # random standard normal
            y = np.random.normal()
            z = np.random.normal()
            v = np.array([x, y, z])

        v = v/np.linalg.norm(v)  # normalize to unit norm
        
        samples.append(v)
        
    return np.array(samples)

# check whether v goes through the polygon with vt_polygon as its vertices
def isinside(v, vt_polygon):
    len_vt = len(vt_polygon)
    list_inside = [np.dot(v,np.cross(vt_polygon[i],vt_polygon[(i+1)%len_vt]))<=0
               for i in range(len_vt)]
    return np.all(list_inside) 

# mean radius and mean squared deviation of radius from the radius of sphere with equivalent volume
def msd_r(Faces, Vertices, Volume, Vect_sample):
    n_sample = len(Vect_sample)
    radius_sphere = (Volume/(4/3*np.pi))**(1/3)
    r_OS = radius_sphere
    
    sigma2 = 0
    r_bar = 0
    # iterate over facets
    for i_faces, face in enumerate(Faces):
        vt_sf_list = Vertices[face['vertices']].tolist()

        # three of the vertices
        v0 = np.array(vt_sf_list[0])
        v1 = np.array(vt_sf_list[1])
        v2 = np.array(vt_sf_list[2])

        # norman vector of this facet    
        n_sf = np.cross((v1-v0),
                        (v1-v2))
        n_sf = n_sf/np.linalg.norm(n_sf)

        index_inside = [isinside(v,vt_sf_list) for v in Vect_sample]
        vect_inside = Vect_sample[index_inside]

        for vect in vect_inside:
            # distance from center to the outer surface
            d_OS = np.abs(np.dot(v0,n_sf)/np.dot(vect,n_sf))

            stretch_OS = d_OS-r_OS

            sigma2 += (stretch_OS**2)
            r_bar += d_OS

    return (sigma2/n_sample), r_bar/n_sample

In [10]:
def msd_coords(c, bounds, n_sample = 2000, Voro=False):
    # Call pyvoro.compute_voronoi function to evaluate the Voronoi tessellations
    voro = pyvoro.compute_voronoi(points,bounds,0.7,periodic=[True]*3)

    list_origin = [v['original'] for v in voro]
    list_volume = [v['volume'] for v in voro]
    list_volume_round = [np.round(v['volume'],decimals=9) for v in voro]
    list_vertices = [v['vertices'] for v in voro]
    list_adjacency = [v['adjacency'] for v in voro]
    list_faces = [v['faces'] for v in voro]
    list_coords = [len(v['faces']) for v in voro]

    # Pick up the unique elements from the list of Voronoi cell volume
    unique_volume, inices, counts = np.unique(list_volume_round,return_counts=True,return_index=True)
    unique_volume_reduced = unique_volume*np.sum(counts)/np.sum(counts*unique_volume)
    unique_adjacency = np.array(list_coords)[inices]

    # Isoperimetric quotient (IQ)
    unique_IQ = []
    for i_cell in range(len(unique_volume)):
        vertices = np.array(list_vertices[inices.tolist()[i_cell]])
        v_cm = np.mean(vertices,axis=0)
        vertices = vertices-v_cm
        faces = list_faces[inices.tolist()[i_cell]]

        area_sum = 0
        for i_faces, face in enumerate(faces):
            vertices_face_list = vertices[face['vertices']].tolist()
            area = area_vertices(vertices_face_list)
            area_sum += area

        unique_IQ.append(36*np.pi*unique_volume[i_cell]**2/area_sum**3)

    IQ_list = np.array(unique_IQ)

    # # scale the unitcell volume such that the averaged Voronoi cell volume = 1
    # unit_cell_volume = np.sum(counts*unique_volume)
    # volume_ratio = np.sum(counts)/unit_cell_volume
    # size_ratio = volume_ratio**(1/3)

    # elastic strain energy
    msd_list = []
    r_bar_list = []
    for i_cell in range(len(unique_volume)):
        # scale each cell that the volume equals 4/3*pi (sphere of radius 1)
        volume_ratio = 4/3*np.pi/unique_volume[i_cell]
        size_ratio = volume_ratio**(1/3)

        # randomly generated radial vectors
        vect_sample = sphere_sample(n_sample)

        # vertices of Voronoi cell
        vertices = np.array(list_vertices[inices.tolist()[i_cell]])*size_ratio
        v_cm = np.mean(vertices,axis=0)
        vertices = vertices-v_cm
        faces = list_faces[inices.tolist()[i_cell]]

        sigma, r_bar = msd_r(Faces=faces, Vertices=vertices, Volume=4/3*np.pi, Vect_sample=vect_sample)
        msd_list.append(sigma)      
        r_bar_list.append(r_bar)   
        
    if Voro:
        return msd_list, r_bar_list, IQ_list, voro
    else:
        return msd_list, r_bar_list, IQ_list

## Test on different polyhedra

### BCC

In [11]:
def unitcell_BCC(origin=[0,0,0], orientation=0):
    '''
    Generate rhombus unit cell
    origin: origin of unitcell, 3*1 array
    orientation: orientation of unitcell, 
                 represented by polar angle of the long diagonl, float
    
    returns: cH = [cH_wht,cH_blu,cH_ylw], lists of coordinates of the 3 types of particles
    '''
    l = 1
        
    R = np.array([[np.cos(orientation),-np.sin(orientation),0],
                  [np.sin(orientation), np.cos(orientation),0],
                  [0,0,1]])

    # place atoms
    c_unit = np.array([[0,0,0],[0,0.5,0.5],[0.5,0,0.5],[0.5,0.5,0]])

    c_unit = np.array([R@c for c in c_unit])+origin

    return c_unit

def unitcell_BCC(origin=[0,0,0], orientation=0):
    '''
    Generate rhombus unit cell
    origin: origin of unitcell, 3*1 array
    orientation: orientation of unitcell, 
                 represented by polar angle of the long diagonl, float
    
    returns: cH = [cH_wht,cH_blu,cH_ylw], lists of coordinates of the 3 types of particles
    '''
    l = 1
        
    R = np.array([[np.cos(orientation),-np.sin(orientation),0],
                  [np.sin(orientation), np.cos(orientation),0],
                  [0,0,1]])

    # place atoms
    c_unit = np.array([[0,0,0],[0.5,0.5,0.5]])

    c_unit = np.array([R@c for c in c_unit])+origin

    return c_unit

def unitcell_SC(origin=[0,0,0], orientation=0):
    '''
    Generate rhombus unit cell
    origin: origin of unitcell, 3*1 array
    orientation: orientation of unitcell, 
                 represented by polar angle of the long diagonl, float
    
    returns: cH = [cH_wht,cH_blu,cH_ylw], lists of coordinates of the 3 types of particles
    '''
    l = 1
        
    R = np.array([[np.cos(orientation),-np.sin(orientation),0],
                  [np.sin(orientation), np.cos(orientation),0],
                  [0,0,1]])

    # place atoms
    c_unit = np.array([[0,0,0]])

    c_unit = np.array([R@c for c in c_unit])+origin

    return c_unit

def unitcell_HCP(origin=[0,0,0], orientation=0, r_ca=np.sqrt(8/3)):
    '''
    Generate rhombus unit cell
    origin: origin of unitcell, 3*1 array
    orientation: orientation of unitcell, 
                 represented by polar angle of the long diagonl, float
    
    returns: cH = [cH_wht,cH_blu,cH_ylw], lists of coordinates of the 3 types of particles
    '''
    l_0 = 1
    l_1 = np.sqrt(3)
    l_2 = r_ca
    
    l = np.array([l_0,l_1,l_2])
        
    R = np.array([[np.cos(orientation),-np.sin(orientation),0],
                  [np.sin(orientation), np.cos(orientation),0],
                  [0,0,1]])

    # place atoms
    c_p = np.array([[0,0,0],[1/2,1/2,0],[0,1/3,1/2],[1/2,5/6,1/2]])
    c_unit = np.array([l*c for c in c_p])
    c_unit = np.array([R@c for c in c_unit])+origin

    return c_unit

In [12]:
# BCC
n_x = 1
n_y = 1
n_layers = 1

# Generate a BCC unit cell
c = unitcell_BCC(origin=[0,0,0], orientation=0)

l = 1.0
bounds = np.array([[0,n_x*l],[0,n_y*l],[0,n_layers*l]])
points = c

msd_BCC, r_bar_BCC, IQ_BCC, voro = msd_coords(c,bounds,n_sample = 500000, Voro=1)
print('msd_BCC = {:0.5f}'.format(msd_BCC[0]))
print('r_bar_BCC = {:0.5f}'.format(r_bar_BCC[0]))
print('IQ_BCC = {:0.5f}'.format(IQ_BCC[0]))

msd_BCC = 0.00422
r_bar_BCC = 0.99587
IQ_BCC = 0.75337


### A15

In [13]:
# A15
r_ca = 1/np.sqrt((1+np.sqrt(3)/2)**2+0.5**2) # cubic lattice
# r_ca = 0.522
n_x = 1
n_y = 1
n_layers = 1
l_c = 1*np.sqrt((1+np.sqrt(3)/2)**2+0.5**2)*r_ca
print(l_c)
    
# Generate a FK A15 primitive unit cell
c_layer_A15 = c_A15(n_x,n_y,Ratio_ca=1/r_ca)
c_rod = stack_coords([shift_coords(c_layer_A15, np.array([0,0,l_c])*s) for s in range(n_layers)])

l = 1.0
bounds = np.array([[0,n_x*l],[0,n_y*l],[0,n_layers*l_c]])
points = np.vstack(c_rod)

msd_A15, r_bar_A15, IQ_A15, voro = msd_coords(points,bounds,n_sample = 500000, Voro=1)
list_origin = [v['original'] for v in voro]
list_volume = [v['volume'] for v in voro]
list_volume_round = [np.round(v['volume'],decimals=9) for v in voro]
list_vertices = [v['vertices'] for v in voro]
list_adjacency = [v['adjacency'] for v in voro]
list_faces = [v['faces'] for v in voro]
list_coords = [len(v['faces']) for v in voro]

# Pick up the unique elements from the list of Voronoi cell volume
unique_volume, inices, counts = np.unique(list_volume_round,return_counts=True,return_index=True)
unique_volume_reduced = unique_volume*np.sum(counts)/np.sum(counts*unique_volume)

print('msd_A15 = {:0.5f}, {:0.5f}'.format(*msd_A15))
print('r_bar_A15 = {:0.5f}, {:0.5f}'.format(*r_bar_A15))
print('IQ_A15 = {:0.5f}, {:0.5f}'.format(*IQ_A15))
print('counts = {:d}, {:d}'.format(*counts))
print('volume = {:0.5f}, {:0.5f}'.format(*unique_volume_reduced))

0.9999999999999999
msd_A15 = 0.00329, 0.00541
r_bar_A15 = 0.99669, 0.99473
IQ_A15 = 0.74931, 0.76589
counts = 2, 6
volume = 0.97656, 1.00781


### sigma

In [14]:
n_x = 1
n_y = 1
n_layers = 1

r_ca = 0.53
l_c = 1*np.sqrt((1+np.sqrt(3)/2)**2+0.5**2)*r_ca
l = np.sqrt((1+np.sqrt(3)/2)**2+0.5**2)

 # Generate a FK sigma primitive unit cell
c_layer_sigma = c_sigma(n_x,n_y,Ratio_ca=1/r_ca)
c_rod = stack_coords([shift_coords(c_layer_sigma, np.array([0,0,l_c])*s) for s in range(n_layers)])

bounds = np.array([[0,n_x*l],[0,n_y*l],[0,n_layers*l_c]])
points = np.vstack(c_rod)

msd_sigma, r_bar_sigma, IQ_sigma, voro = msd_coords(c,bounds,n_sample = 500000, Voro=1)
list_origin = [v['original'] for v in voro]
list_volume = [v['volume'] for v in voro]
list_volume_round = [np.round(v['volume'],decimals=9) for v in voro]
list_vertices = [v['vertices'] for v in voro]
list_adjacency = [v['adjacency'] for v in voro]
list_faces = [v['faces'] for v in voro]
list_coords = [len(v['faces']) for v in voro]

# Pick up the unique elements from the list of Voronoi cell volume
unique_volume, inices, counts = np.unique(list_volume_round,return_counts=True,return_index=True)
unique_volume_reduced = unique_volume*np.sum(counts)/np.sum(counts*unique_volume)

print('msd_sigma = {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}'.format(*msd_sigma))
print('r_bar_sigma = {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}'.format(*r_bar_sigma))
print('IQ_sigma = {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}'.format(*IQ_sigma))
print('counts = {:d}, {:d}, {:d}, {:d}, {:d}'.format(*counts))
print('volume = {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}, {:0.5f}'.format(*unique_volume_reduced))

msd_sigma = 0.00638, 0.00440, 0.00496, 0.00507, 0.00506
r_bar_sigma = 0.99363, 0.99551, 0.99515, 0.99504, 0.99514
IQ_sigma = 0.74149, 0.74396, 0.76962, 0.77392, 0.76719
counts = 2, 8, 8, 4, 8
volume = 0.89621, 0.94070, 0.99620, 1.03101, 1.07354
